# シミュレーション実験

In [1]:
import sys
sys.path.append('../')

from scripts.simulator import Transaction, StableTransaction, JCT,StableJCT
from scripts.price_data.get_price import GetPriceData

price_getter = GetPriceData()

In [2]:
jct_portfolio_A = {
    'JPY': {
        'num': 10000000,
        'is_usd': False
    },
    '8411.T': {
        'num': 10000,
        'is_usd': False
    },
}

jct_portfolio_B = {
    'JPY': {
        'num': 10000000,
        'is_usd': False
    }
}

st_portfolio = {
    'MSFT': {
        'num': 100,
        'is_usd': True
    },
    '3967.T': {
        'num': 10000,
        'is_usd': False
    },
    '6578.T': {
        'num': 10000,
        'is_usd': False
    },
}

## 1. JCTが可変の場合

In [3]:
# margin call ratio = 0.8

from datetime import date, timedelta

def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
jct_ex = JCT()

jct_ex.add_jct('A', jct_portfolio_A, '2021-06-20')
jct_ex.add_jct('B', jct_portfolio_B, '2021-06-20')
jct_ex.update_price("2021-06-21")
print('jct users info...')
print(jct_ex.users)

transaction_var = Transaction('A', 'B', st_portfolio, jct_ex.get_jct_price(), "2021-06-21", 1.05, 0.8)

print('='*50)
print('Start simulation...')
print('='*50)
for date in date_range(date(2021, 6, 22), date(2021, 7, 10)):
    print(date)
    jct_ex.update_price(date)
    jct_price = jct_ex.get_jct_price()
    print('jct price: ', jct_price)
    transaction_var.update_st_price(date)
    diff = transaction_var.check_diff_and_margin_call(jct_price)
    if diff is not None:
        if diff['from_borrower']:
            if jct_ex.users[diff['from']]['total_jct_num'] < diff['jct_num'] + transaction_var.get_transaction_jct():
                print(f'Warning! {diff["from"]} needs to add margin.')
                print('¥' * 50)
                break
            transaction_var.add_transaction_jct(diff['jct_num'])
            jct_ex.move_jct(diff['from'], diff['to'], diff['jct_num'])
        else:
            if jct_ex.users[diff['from']]['total_jct_num'] < diff['jct_num']:
                print(f'Warning! {diff["from"]} needs to add margin.')
                print('¥' * 50)
                break
            transaction_var.add_transaction_jct(-diff['jct_num'])
            jct_ex.move_jct(diff['from'], diff['to'], diff['jct_num'])
    print('='*50)
print('Finished.')
print(jct_ex.users)

JPY: 1.0
8411.T: 1564.5
jct users info...
{'A': {'JPY': {'num': 10000000, 'is_usd': False}, '8411.T': {'num': 10000, 'is_usd': False}, 'total_jct_num': 25645000.0}, 'B': {'JPY': {'num': 10000000, 'is_usd': False}, 'total_jct_num': 10000000.0}}
Initial jct num is 18925419.26334577.
Start simulation...
2021-06-22
JPY: 1.0
8411.T: 1607.0
jct price:  1.0119231308738954
jct_total: 19151069.514065985
st_total: 18274208.822234068
OK. No margin call.
2021-06-23
JPY: 1.0
8411.T: 1597.0
jct price:  1.0091176883153317
jct_total: 19097975.337425932
st_total: 18270540.19637201
OK. No margin call.
2021-06-24
JPY: 1.0
8411.T: 1604.5
jct price:  1.0112217702342545
jct_total: 19137795.969905972
st_total: 18185318.51862201
OK. No margin call.
2021-06-25
JPY: 1.0
8411.T: 1609.5
jct price:  1.0126244915135363
jct_total: 19164343.058225997
st_total: 18250579.08277423
OK. No margin call.
2021-06-26
JPY: 1.0
8411.T: 1622.5
jct price:  1.016271566839669
jct_total: 19233365.487858057
st_total: 18259628.1570539

In [4]:
# margin call ratio = 0.95

from datetime import date, timedelta

def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
jct_ex = JCT()

jct_ex.add_jct('A', jct_portfolio_A, '2021-06-20')
jct_ex.add_jct('B', jct_portfolio_B, '2021-06-20')
jct_ex.update_price("2021-06-21")
print('jct users info...')
print(jct_ex.users)

transaction_var = Transaction('A', 'B', st_portfolio, jct_ex.get_jct_price(), "2021-06-21", 1.05, 0.95)

print('='*50)
print('Start simulation...')
print('='*50)
for date in date_range(date(2021, 6, 22), date(2021, 7, 10)):
    print(date)
    jct_ex.update_price(date)
    jct_price = jct_ex.get_jct_price()
    print('jct price: ', jct_price)
    transaction_var.update_st_price(date)
    diff = transaction_var.check_diff_and_margin_call(jct_price)
    if diff is not None:
        if diff['from_borrower']:
            if jct_ex.users[diff['from']]['total_jct_num'] < diff['jct_num'] + transaction_var.get_transaction_jct():
                print(f'Warning! {diff["from"]} needs to add margin.')
                print('¥' * 50)
                break
            transaction_var.add_transaction_jct(diff['jct_num'])
            jct_ex.move_jct(diff['from'], diff['to'], diff['jct_num'])
        else:
            if jct_ex.users[diff['from']]['total_jct_num'] < diff['jct_num']:
                print(f'Warning! {diff["from"]} needs to add margin.')
                print('¥' * 50)
                break
            transaction_var.add_transaction_jct(-diff['jct_num'])
            jct_ex.move_jct(diff['from'], diff['to'], diff['jct_num'])
    print('='*50)
print('Finished.')
print(jct_ex.users)

JPY: 1.0
8411.T: 1564.5
jct users info...
{'A': {'JPY': {'num': 10000000, 'is_usd': False}, '8411.T': {'num': 10000, 'is_usd': False}, 'total_jct_num': 25645000.0}, 'B': {'JPY': {'num': 10000000, 'is_usd': False}, 'total_jct_num': 10000000.0}}
Initial jct num is 18925419.26334577.
Start simulation...
2021-06-22
JPY: 1.0
8411.T: 1607.0
jct price:  1.0119231308738954
jct_total: 19151069.514065985
st_total: 18274208.822234068
OK. No margin call.
2021-06-23
JPY: 1.0
8411.T: 1597.0
jct price:  1.0091176883153317
jct_total: 19097975.337425932
st_total: 18270540.19637201
OK. No margin call.
2021-06-24
JPY: 1.0
8411.T: 1604.5
jct price:  1.0112217702342545
jct_total: 19137795.969905972
st_total: 18185318.51862201
OK. No margin call.
2021-06-25
JPY: 1.0
8411.T: 1609.5
jct price:  1.0126244915135363
jct_total: 19164343.058225997
st_total: 18250579.08277423
OK. No margin call.
2021-06-26
JPY: 1.0
8411.T: 1622.5
jct price:  1.016271566839669
jct_total: 19233365.487858057
st_total: 18259628.1570539

In [5]:
# margin call ratio = 0.99

from datetime import date, timedelta

def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)
        
jct_ex = JCT()

jct_ex.add_jct('A', jct_portfolio_A, '2021-06-20')
jct_ex.add_jct('B', jct_portfolio_B, '2021-06-20')
jct_ex.update_price("2021-06-21")
print('jct users info...')
print(jct_ex.users)

transaction_var = Transaction('A', 'B', st_portfolio, jct_ex.get_jct_price(), "2021-06-21", 1.05, 0.99)

print('='*50)
print('Start simulation...')
print('='*50)
for date in date_range(date(2021, 6, 22), date(2021, 7, 10)):
    print(date)
    jct_ex.update_price(date)
    jct_price = jct_ex.get_jct_price()
    print('jct price: ', jct_price)
    transaction_var.update_st_price(date)
    diff = transaction_var.check_diff_and_margin_call(jct_price)
    if diff is not None:
        if diff['from_borrower']:
            if jct_ex.users[diff['from']]['total_jct_num'] < diff['jct_num'] + transaction_var.get_transaction_jct():
                print(f'Warning! {diff["from"]} needs to add margin.')
                print('¥' * 50)
                break
            transaction_var.add_transaction_jct(diff['jct_num'])
            jct_ex.move_jct(diff['from'], diff['to'], diff['jct_num'])
        else:
            if jct_ex.users[diff['from']]['total_jct_num'] < diff['jct_num']:
                print(f'Warning! {diff["from"]} needs to add margin.')
                print('¥' * 50)
                break
            transaction_var.add_transaction_jct(-diff['jct_num'])
            jct_ex.move_jct(diff['from'], diff['to'], diff['jct_num'])
    print('='*50)
print('Finished.')
print(jct_ex.users)

JPY: 1.0
8411.T: 1564.5
jct users info...
{'A': {'JPY': {'num': 10000000, 'is_usd': False}, '8411.T': {'num': 10000, 'is_usd': False}, 'total_jct_num': 25645000.0}, 'B': {'JPY': {'num': 10000000, 'is_usd': False}, 'total_jct_num': 10000000.0}}
Initial jct num is 18925419.26334577.
Start simulation...
2021-06-22
JPY: 1.0
8411.T: 1607.0
jct price:  1.0119231308738954
jct_total: 19151069.514065985
st_total: 18274208.822234068
Margin Call! Lender needs to add 677274.7610773453 or more JCT!
move 677274.7610773453 from B to A
{'A': {'JPY': {'num': 10000000, 'is_usd': False}, '8411.T': {'num': 10000, 'is_usd': False}, 'total_jct_num': 26322274.761077344}, 'B': {'JPY': {'num': 10000000, 'is_usd': False}, 'total_jct_num': 9322725.238922656}}
2021-06-23
JPY: 1.0
8411.T: 1597.0
jct price:  1.0091176883153317
jct_total: 18414525.396173242
st_total: 18270540.19637201
OK. No margin call.
2021-06-24
JPY: 1.0
8411.T: 1604.5
jct price:  1.0112217702342545
jct_total: 18452920.987074357
st_total: 1818531

## 2. JCTが固定（1JCT = 1円）の場合

In [6]:
# margin call ratio = 0.8
from datetime import date, timedelta

def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

jct_stb = StableJCT()

jct_stb.add_jct('A', jct_portfolio_A, '2021-06-20')
jct_stb.add_jct('B', jct_portfolio_B, '2021-06-20')
changes = jct_stb.get_change('2021-06-21')
for user, change_num in changes.items():
    jct_stb.exec_change_num(user, change_num)
print('jct users info...')
print(jct_stb.users)

transaction_stb = StableTransaction('A', 'B', st_portfolio, "2021-06-21", 1.05, 0.8)

print('='*50)
print('Start simulation...')
print('='*50)
for date in date_range(date(2021, 6, 22), date(2021, 7, 10)):
    print(date)

    changes = jct_stb.get_change(date)
    for user, change_num in changes.items():
        if user == transaction_stb.borrower and (jct_stb.users[user]['total_jct_num'] + change_num < transaction_stb.get_transaction_jct()):
            print(f'Warning! {user} needs to add margin.')
            print('Cannot delete JCT.')
            print('transaction_jct_num: ', transaction_stb.get_transaction_jct())
            print('total_jct_num: ', jct_stb.users[user]['total_jct_num'] + change_num)
            print('¥' * 50)
            break
        jct_stb.exec_change_num(user, change_num)

    transaction_stb.update_st_price(date)
    
    diff = transaction_stb.check_diff_and_margin_call()
    if diff is not None:
        if diff['from_borrower']:
            if jct_stb.users[diff['from']]['total_jct_num'] < diff['jct_num'] + transaction_stb.get_transaction_jct():
                print(f'Warning! {diff["from"]} needs to add margin.')
                print('¥' * 50)
                break
            transaction_stb.add_transaction_jct(diff['jct_num'])
            jct_stb.move_jct(diff['from'], diff['to'], diff['jct_num'])
        else:
            if jct_stb.users[diff['from']]['total_jct_num'] < diff['jct_num']:
                print(f'Warning! {diff["from"]} needs to add margin.')
                print('¥' * 50)
                break
            transaction_stb.add_transaction_jct(-diff['jct_num'])
            jct_stb.move_jct(diff['from'], diff['to'], diff['jct_num'])
    print('='*50)
print('Finished.')

JPY: 1.0
8411.T: 1564.5
JPY: 1.0
jct users info...
{'A': {'JPY': {'num': 10000000, 'is_usd': False, 'price': 1.0}, '8411.T': {'num': 10000, 'is_usd': False, 'price': 1564.5}, 'total_jct_num': 25645000.0}, 'B': {'JPY': {'num': 10000000, 'is_usd': False, 'price': 1.0}, 'total_jct_num': 10000000.0}}
Initial jct num is 18925419.26334577.
Start simulation...
2021-06-22
JPY: 1.0
8411.T: 1607.0
JPY: 1.0
OK. No margin call.
2021-06-23
JPY: 1.0
8411.T: 1597.0
JPY: 1.0
OK. No margin call.
2021-06-24
JPY: 1.0
8411.T: 1604.5
JPY: 1.0
OK. No margin call.
2021-06-25
JPY: 1.0
8411.T: 1609.5
JPY: 1.0
OK. No margin call.
2021-06-26
JPY: 1.0
8411.T: 1622.5
JPY: 1.0
OK. No margin call.
2021-06-27
JPY: 1.0
8411.T: 1622.5
JPY: 1.0
OK. No margin call.
2021-06-28
JPY: 1.0
8411.T: 1622.5
JPY: 1.0
OK. No margin call.
2021-06-29
JPY: 1.0
8411.T: 1596.0
JPY: 1.0
OK. No margin call.
2021-06-30
JPY: 1.0
8411.T: 1587.5
JPY: 1.0
OK. No margin call.
2021-07-01
JPY: 1.0
8411.T: 1587.5
JPY: 1.0
OK. No margin call.
2021

In [7]:
# margin call ratio = 0.95
from datetime import date, timedelta

def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

jct_stb = StableJCT()

jct_stb.add_jct('A', jct_portfolio_A, '2021-06-20')
jct_stb.add_jct('B', jct_portfolio_B, '2021-06-20')
changes = jct_stb.get_change('2021-06-21')
for user, change_num in changes.items():
    jct_stb.exec_change_num(user, change_num)
print('jct users info...')
print(jct_stb.users)

transaction_stb = StableTransaction('A', 'B', st_portfolio, "2021-06-21", 1.05, 0.95)

print('='*50)
print('Start simulation...')
print('='*50)
for date in date_range(date(2021, 6, 22), date(2021, 7, 10)):
    print(date)

    changes = jct_stb.get_change(date)
    for user, change_num in changes.items():
        if user == transaction_stb.borrower and (jct_stb.users[user]['total_jct_num'] + change_num < transaction_stb.get_transaction_jct()):
            print(f'Warning! {user} needs to add margin.')
            print('Cannot delete JCT.')
            print('transaction_jct_num: ', transaction_stb.get_transaction_jct())
            print('total_jct_num: ', jct_stb.users[user]['total_jct_num'] + change_num)
            print('¥' * 50)
            break
        jct_stb.exec_change_num(user, change_num)

    transaction_stb.update_st_price(date)
    
    diff = transaction_stb.check_diff_and_margin_call()
    if diff is not None:
        if diff['from_borrower']:
            if jct_stb.users[diff['from']]['total_jct_num'] < diff['jct_num'] + transaction_stb.get_transaction_jct():
                print(f'Warning! {diff["from"]} needs to add margin.')
                print('¥' * 50)
                break
            transaction_stb.add_transaction_jct(diff['jct_num'])
            jct_stb.move_jct(diff['from'], diff['to'], diff['jct_num'])
        else:
            if jct_stb.users[diff['from']]['total_jct_num'] < diff['jct_num']:
                print(f'Warning! {diff["from"]} needs to add margin.')
                print('¥' * 50)
                break
            transaction_stb.add_transaction_jct(-diff['jct_num'])
            jct_stb.move_jct(diff['from'], diff['to'], diff['jct_num'])
    print('='*50)
print('Finished.')

JPY: 1.0
8411.T: 1564.5
JPY: 1.0
jct users info...
{'A': {'JPY': {'num': 10000000, 'is_usd': False, 'price': 1.0}, '8411.T': {'num': 10000, 'is_usd': False, 'price': 1564.5}, 'total_jct_num': 25645000.0}, 'B': {'JPY': {'num': 10000000, 'is_usd': False, 'price': 1.0}, 'total_jct_num': 10000000.0}}
Initial jct num is 18925419.26334577.
Start simulation...
2021-06-22
JPY: 1.0
8411.T: 1607.0
JPY: 1.0
OK. No margin call.
2021-06-23
JPY: 1.0
8411.T: 1597.0
JPY: 1.0
OK. No margin call.
2021-06-24
JPY: 1.0
8411.T: 1604.5
JPY: 1.0
OK. No margin call.
2021-06-25
JPY: 1.0
8411.T: 1609.5
JPY: 1.0
OK. No margin call.
2021-06-26
JPY: 1.0
8411.T: 1622.5
JPY: 1.0
OK. No margin call.
2021-06-27
JPY: 1.0
8411.T: 1622.5
JPY: 1.0
OK. No margin call.
2021-06-28
JPY: 1.0
8411.T: 1596.0
JPY: 1.0
OK. No margin call.
2021-06-29
JPY: 1.0
8411.T: 1596.0
JPY: 1.0
OK. No margin call.
2021-06-30
JPY: 1.0
8411.T: 1587.5
JPY: 1.0
OK. No margin call.
2021-07-01
JPY: 1.0
8411.T: 1587.5
JPY: 1.0
Margin Call! Lender need

In [8]:
# margin call ratio = 0.99
from datetime import date, timedelta

def date_range(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

jct_stb = StableJCT()

jct_stb.add_jct('A', jct_portfolio_A, '2021-06-20')
jct_stb.add_jct('B', jct_portfolio_B, '2021-06-20')
changes = jct_stb.get_change('2021-06-21')
for user, change_num in changes.items():
    jct_stb.exec_change_num(user, change_num)
print('jct users info...')
print(jct_stb.users)

transaction_stb = StableTransaction('A', 'B', st_portfolio, "2021-06-21", 1.05, 0.99)

print('='*50)
print('Start simulation...')
print('='*50)
for date in date_range(date(2021, 6, 22), date(2021, 7, 10)):
    print(date)

    changes = jct_stb.get_change(date)
    for user, change_num in changes.items():
        if user == transaction_stb.borrower and (jct_stb.users[user]['total_jct_num'] + change_num < transaction_stb.get_transaction_jct()):
            print(f'Warning! {user} needs to add margin.')
            print('Cannot delete JCT.')
            print('transaction_jct_num: ', transaction_stb.get_transaction_jct())
            print('total_jct_num: ', jct_stb.users[user]['total_jct_num'] + change_num)
            print('¥' * 50)
            break
        jct_stb.exec_change_num(user, change_num)

    transaction_stb.update_st_price(date)
    
    diff = transaction_stb.check_diff_and_margin_call()
    if diff is not None:
        if diff['from_borrower']:
            if jct_stb.users[diff['from']]['total_jct_num'] < diff['jct_num'] + transaction_stb.get_transaction_jct():
                print(f'Warning! {diff["from"]} needs to add margin.')
                print('¥' * 50)
                break
            transaction_stb.add_transaction_jct(diff['jct_num'])
            jct_stb.move_jct(diff['from'], diff['to'], diff['jct_num'])
        else:
            if jct_stb.users[diff['from']]['total_jct_num'] < diff['jct_num']:
                print(f'Warning! {diff["from"]} needs to add margin.')
                print('¥' * 50)
                break
            transaction_stb.add_transaction_jct(-diff['jct_num'])
            jct_stb.move_jct(diff['from'], diff['to'], diff['jct_num'])
    print('='*50)
print('Finished.')

JPY: 1.0
8411.T: 1564.5
JPY: 1.0
jct users info...
{'A': {'JPY': {'num': 10000000, 'is_usd': False, 'price': 1.0}, '8411.T': {'num': 10000, 'is_usd': False, 'price': 1564.5}, 'total_jct_num': 25645000.0}, 'B': {'JPY': {'num': 10000000, 'is_usd': False, 'price': 1.0}, 'total_jct_num': 10000000.0}}
Initial jct num is 18925419.26334577.
Start simulation...
2021-06-22
JPY: 1.0
8411.T: 1607.0
JPY: 1.0
Margin Call! Lender needs to add 461956.248478245 or more JCT!
move 461956.248478245 from B to A
{'A': {'JPY': {'num': 10000000, 'is_usd': False, 'price': 1.0}, '8411.T': {'num': 10000, 'is_usd': False, 'price': 1607.0}, 'total_jct_num': 26531956.248478245}, 'B': {'JPY': {'num': 10000000, 'is_usd': False, 'price': 1.0}, 'total_jct_num': 9538043.751521755}}
2021-06-23
JPY: 1.0
8411.T: 1596.0
JPY: 1.0
Margin Call! Lender needs to add 168288.18834684044 or more JCT!
move 168288.18834684044 from B to A
{'A': {'JPY': {'num': 10000000, 'is_usd': False, 'price': 1.0}, '8411.T': {'num': 10000, 'is_usd